count	非 NA 值的数量
describe	针对 Series 或 DF 的列计算汇总统计
min , max	最小值和最大值
argmin , argmax	最小值和最大值的索引位置（整数）
idxmin , idxmax	最小值和最大值的索引值
quantile	样本分位数（0 到 1）
sum	求和
mean	均值
median	中位数
mad	根据均值计算平均绝对离差
var	方差
std	标准差
skew	样本值的偏度（三阶矩）
kurt	样本值的峰度（四阶矩）
cumsum	样本值的累计和
cummin , cummax	样本值的累计最大值和累计最小值
cumprod	样本值的累计积
diff	计算一阶差分（对时间序列很有用）
pct_change	计算百分数变化


In [7]:
# 引入这个项目需要的库
import numpy as np
import pandas as pd
#import visuals as vs
from IPython.display import display # 使得我们可以对DataFrame使用display()函数
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.decomposition import RandomizedPCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.grid_search import GridSearchCV
from scipy import stats

import time

# 设置以内联的形式显示matplotlib绘制的图片（在notebook中显示更美观）
%matplotlib inline

# 载入整个数据集
try:
    data_origin = pd.read_csv("first_train_index_20180131.csv")
    #data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    print "dataset has {} samples with {} features each.".format(*data_origin.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"

dataset has 483851 samples with 2 features each.


In [8]:
print data_origin['type'].value_counts()

star       442969
unknown     34288
galaxy       5231
qso          1363
Name: type, dtype: int64


In [10]:
#抽样,4000数据集
data_star = data_origin[data_origin['type']== 'star'].sample(n=2000,random_state=0)
data_unknown = data_origin[data_origin['type']== 'unknown'].sample(n=2000,random_state=0)
data_galaxy = data_origin[data_origin['type']== 'galaxy'].sample(n=2000,random_state=0)
data_qso = data_origin[data_origin['type']== 'qso'].sample(n=1300,random_state=0)
data = pd.concat([data_star,data_unknown,data_galaxy,data_qso])

#设置index，提高检索效率
data = data.set_index('id', drop=True, append=False, inplace=False, verify_integrity=False)

In [12]:
#读取data_id对应波长文件

#四舍五入
import time

'''
def ggg(x):
    x = float(x)
    #x = x/10
    return round(x,2)
'''

waves_counts = {}
count = 0
waves = {}
all_peaks = {}
wave_range = {}

'''
for p in ('max','min','mean','std','kurt','unique'):
    waves[p] = {}
n = 10
#for j in range(1,n):
    #waves[j] = {}
'''

for i in data.index:
    f = open('/Users/suyangluojia/Desktop/LEARN/天池/天文数据挖掘/first_train_data_20180131/%s.txt'%i, 'r')
    waves_str = f.read()
    waves_s = waves_str.split(',')
    waves_float = pd.Series(map(float,waves_s))
    #waves_int = pd.Series(map(ggg,waves_s)).value_counts()
    
    
    peaks = {}
    kernel = stats.gaussian_kde(waves_float)
    
    '''
    if int(min(waves_float))> -10000:
        wave_min = int(min(waves_float))
    else: 
        wave_min = -10000
        
    if int(max(waves_float))< 10000:
        wave_max = int(max(waves_float))
    else: 
        wave_max = 10000
    '''
    
        
    x = list(np.arange(-20000,20000,2))+list(np.arange(20000,100000,200))
    peaks = pd.Series(dict(zip(x, kernel(x)))).sort_values(ascending = False)
    all_peaks[i] = peaks
    
    
    '''
    #波长各个统计值
    waves['max'][i] = max(waves_float)
    waves['min'][i] = min(waves_float)
    waves['mean'][i] = waves_float.mean()
    waves['std'][i] = waves_float.std()
    waves['kurt'][i] = waves_float.kurt()
    waves['unique'][i] = len(waves_float.unique())
    #waves['peaks_count'][i] = len(peaks)
    '''
    
    '''
    peaks_index = list(peaks.index)
    if len(peaks) >= 10:
        waves['peaks'][i] = np.array(peaks_index[:10])
    else:
        for c in range(len(peaks),10):
            peaks_index.append(0)
        waves['peaks'][i] = np.array(peaks_index[:10])
    '''
    
    '''
    #前n个波长众数
    for j in range(1,n):
        try:
            waves[j][i] = peaks.index[j]
        except:
            waves[j][i] = np.nan
            #print i,waves_int
    '''

    
    f.close()
    count +=1
    print count,i
    wave_range[i] = (int(min(waves_float)),int(max(waves_float)))

1 1089807
2 1348326
3 1176902
4 927833
5 1181221
6 719480
7 866392
8 702287
9 1123890
10 1333483
11 701287
12 934607
13 1027406
14 1077806
15 716513
16 1022735
17 1322236
18 912719
19 851516
20 861882
21 867034
22 980449
23 1088680
24 1309903
25 1221681
26 766490
27 883557
28 770081
29 979927
30 744819
31 1165796
32 1079527
33 873859
34 1268825
35 1008028
36 1146099
37 921801
38 1132073
39 1207302
40 1106124
41 824617
42 994538
43 1037808
44 860227
45 1175715
46 955667
47 1184362
48 1124091
49 1336677
50 743418
51 1021105
52 1296361
53 763789
54 836534
55 1364645
56 994092
57 1097919
58 721533
59 908657
60 1202387
61 1049429
62 1245371
63 1346191
64 1016111
65 1111998
66 945307
67 1223782
68 1294040
69 976450
70 1108909
71 982852
72 753295
73 1264646
74 1159991
75 1290592
76 1080128
77 863210
78 1011037
79 1111314
80 853742
81 1161380
82 758026
83 1197579
84 796684
85 1203302
86 741068
87 1051467
88 1187360
89 961384
90 1134781
91 739238
92 996305
93 981335
94 1330950
95 972057
96 7653

In [13]:
data_ooo = pd.DataFrame(all_peaks)
columns = data_ooo.columns
index = data_ooo.index
data_train_x = np.matrix(data_ooo.as_matrix(columns=None)).T
data_train_x = pd.DataFrame(data_train_x, index = columns, columns = index)
data_train_y = data['type'][data_train_x.index]

In [14]:
wave_range

{688132: (-12, 424),
 786438: (-10, 177),
 1245196: (-72, 106),
 917520: (-7, 63),
 819217: (-49, 181),
 884755: (-2, 122),
 1048599: (-20, 56),
 983067: (3, 210),
 1212447: (-760, 1511),
 720931: (-106, 119),
 1343527: (-30, 47),
 1310765: (-47, 142),
 1343542: (-211, 250),
 753721: (-23, 38),
 1081417: (10, 345),
 720973: (37, 213),
 884829: (-7, 21),
 950366: (362, 2540),
 1179664: (2, 102),
 1212516: (0, 2230),
 1081446: (252, 1662),
 1343600: (-1219, 2071),
 1114225: (-18, 63),
 1343610: (-35, 76),
 753788: (514, 3410),
 1147009: (185, 1891),
 1048711: (1, 205),
 1081488: (-36, 34),
 939374: (-24, 15),
 1179801: (-38, 291),
 786586: (2244, 9128),
 1004912: (-46, 37),
 1081516: (-58, 66),
 1114290: (-15, 44),
 786615: (44107, 336461),
 753848: (182, 1779),
 1245371: (8, 64),
 1016003: (3, 72),
 1147079: (-38, 23),
 1081556: (2, 256),
 721110: (0, 61),
 721115: (-86, 166),
 1245405: (-5, 11),
 1212638: (1765, 8241),
 1048804: (-232, 287),
 950504: (-20, 56),
 819435: (-51, 71),
 884

In [15]:
len(data_train_y)

7300

In [16]:
#训练数据集
X_train, X_test, y_train, y_test = train_test_split(data_train_x, data_train_y, test_size=0.2, random_state=0)

In [26]:
#PCA
# TODO：通过在good_data上使用PCA，将其转换成和当前特征数一样多的维度
#pca = PCA(n_components=100)
#pca.fit(good_data)
# TODO：使用上面的PCA拟合将变换施加在log_samples上
#pca_samples = pca.transform(samples)
# 生成PCA的结果图
#pca_results = vs.pca_results(good_data, pca)

n_components = 30
print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[1])
t0 = time.time()
pca = PCA(n_components=n_components, whiten=True).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print "done in %0.3fs" % (time.time() - t0)
print pca.explained_variance_ratio_

Extracting the top 30 eigenfaces from 20400 faces
done in 6.819s
[ 0.31488289  0.17948142  0.10890923  0.08212754  0.06170586  0.04929677
  0.03733478  0.02802309  0.0218734   0.017886    0.01424915  0.01185644
  0.01089925  0.00883862  0.0076193   0.00620006  0.00522486  0.00411931
  0.00343407  0.00295411  0.00259534  0.00242395  0.00204101  0.00178878
  0.00161625  0.00132135  0.00116903  0.00105918  0.00102725  0.00085037]


In [30]:
from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import f1_score

print "Fitting the classifier to the training set"
t0 = time.time()
param_grid = {
         'alpha': [1e-3, 5e-3, 1e-4, 5e-4, 1e-5],
          'hidden_layer_sizes':[(8,),(10,),(12,),(14,),(16,),(18,),(20,)],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
clf_1 = MLPClassifier(solver='lbfgs', alpha=1e-3,hidden_layer_sizes=(15,), random_state=0)
#clf_1 = GridSearchCV(MLPClassifier(solver='lbfgs'), param_grid)
clf_1 = clf_1.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time.time() - t0)
#print "Best estimator found by grid search:"
#print clf_1.best_estimator_

y_pred_1 = clf_1.predict(X_test_pca)
results_1 = accuracy_score(y_test,  y_pred_1)
f1_score_1 = f1_score(y_test,  y_pred_1, average='macro') 
print results_1,f1_score_1

Fitting the classifier to the training set
done in 0.823s
0.685616438356 0.680440330087


In [31]:
#混淆矩阵
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_1)

array([[277,  39,  32,  55],
       [ 26, 165,   3,  77],
       [ 55,   6, 304,  37],
       [ 30,  55,  44, 255]])

In [32]:
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score,f1_score

print "Fitting the classifier to the training set"
t0 = time.time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5, 1e6], 
         #'degree': [2,3,4],
         'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
#clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = SVC(kernel='rbf', class_weight='balanced', C = 5000, gamma = 1)
clf = clf.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time.time() - t0)
#print "Best estimator found by grid search:"
#print clf.best_estimator_

y_pred = clf.predict(X_test_pca)
results = accuracy_score(y_test,  y_pred)
f1_score = f1_score(y_test,  y_pred, average='macro')
results,f1_score

Fitting the classifier to the training set
done in 6.023s


(0.60068493150684932, 0.58621731514143871)

In [3]:
#读取测试文件，并封装在各个文件夹中
try:
    data_sub = pd.read_csv("first_test_index_20180131.csv")
    print data_sub
    #data.drop(['Region', 'Channel'], axis = 1, inplace = True)
    #print "dataset has {} samples with {} features each.".format(*data_origin.shape)
except:
    print "Dataset could not be loaded. Is the dataset missing?"

data_sub = data_sub.set_index('id', drop=True, append=False, inplace=False, verify_integrity=False)


all_peaks_sub = {}
count = 0
dataframe = {}
data_sub_x = {}

for c in np.arange(73000,100000,1000):
    all_peaks_sub[c] = {}
    for i in data_sub.index[c:c+1000]:
        f = open('/Users/suyangluojia/Desktop/LEARN/天池/天文数据挖掘/first_test_data_20180131/%s.txt'%i, 'r')
        d = open('/Users/suyangluojia/Desktop/LEARN/天池/天文数据挖掘/peaks.txt', 'a')
        waves_str = f.read()
        waves_s = waves_str.split(',')
        waves_float = pd.Series(map(float,waves_s))
        kernel = stats.gaussian_kde(waves_float)
        
        x = list(np.arange(-20000,20000,2))+list(np.arange(20000,100000,200))
        peaks = pd.Series(dict(zip(x, kernel(x)))).sort_values(ascending = False)
        all_peaks_sub[c][i] = peaks
        #print peaks
        #print all_peaks_sub[0]
    
        f.close()
        count +=1
        print count,i  
    dataframe[c] = pd.DataFrame(all_peaks_sub[c])
    index = dataframe[c].index
    columns = dataframe[c].columns
    data_sub_x[c] = np.matrix(dataframe[c].as_matrix(columns=None)).T
    data_sub_x[c] = pd.DataFrame(data_sub_x[c], index = columns, columns = index)
    data_sub_x[c].to_csv('data_sub_x_%d.csv'%c)
    print '%d has done'%c

            id
0       822405
1      1142964
2      1241977
3      1309049
4      1085810
5      1366842
6       725968
7      1116472
8       882715
9       735536
10      836089
11      905006
12      837335
13     1083177
14      894316
15     1332900
16     1353413
17     1251998
18      834783
19     1006689
20     1288742
21      812148
22     1040844
23      801315
24     1138272
25      906913
26     1326674
27      891878
28      848529
29     1259595
...        ...
99970  1270414
99971  1314982
99972   996410
99973  1129956
99974  1086699
99975   906496
99976  1217147
99977  1336344
99978   868674
99979  1207339
99980  1104884
99981  1314562
99982  1011480
99983   753907
99984   758906
99985  1036856
99986  1005019
99987   753140
99988  1030548
99989   726270
99990   920741
99991   877839
99992  1163646
99993   779167
99994  1220669
99995  1231823
99996  1265571
99997   828206
99998  1002346
99999  1027574

[100000 rows x 1 columns]
1 1245468
2 1209614
3 856018
4 1357897
5 91

In [40]:
dataframe = {}
for c in np.arange(0,100000,1000):
    print c
    dataframe[c] = pd.read_csv('data_sub_x_%d.csv'%c)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [43]:
data_sub = pd.DataFrame()
X_sub_pca = {}
y_pred_sub = {}
for i in np.arange(0,1000,1000):
    dataframe[i] = dataframe[i].set_index('Unnamed: 0', drop=True, append=False, inplace=False, verify_integrity=False)
    X_sub_pca[i] = pca.transform(dataframe[i])
    y_pred_sub[i] = clf_1.predict(X_sub_pca[i])
    dataframe[i]['y_pred'] = y_pred_sub[i]
    dataframe[i] = dataframe[i].loc[:,'y_pred']
    print i

AttributeError: 'Series' object has no attribute 'set_index'

In [45]:
data_sub = pd.Series()
for i in np.arange(0,100000,1000):
    data_sub = data_sub.append(dataframe[i])
    print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [47]:
len(data_sub)

100000

In [48]:
data_sub.to_csv('submission.csv',header = False)